## Photometry on MIRI images

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import warnings
import subprocess
import json
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

from astropy.io import fits
from astropy.wcs import WCS, FITSFixedWarning
from astropy.table import Table
import astropy.units as u
from astropy.stats import SigmaClip, sigma_clip, sigma_clipped_stats
from astropy.visualization import simple_norm

from photutils.aperture import EllipticalAperture, aperture_photometry
from photutils.background import Background2D, MedianBackground

warnings.simplefilter("ignore", category=FITSFixedWarning)


cutout_dir = "/home/bpc/University/master/Red_Cardinal/cutouts/"
phot_dir = "/home/bpc/University/master/Red_Cardinal/photometry/"


# Section to obtain modified apertures

Let's inspect Amirs table:

In [ ]:
table_path =  '/home/bpc/University/master/Red_Cardinal/Flux_Aperture_PSFMatched_AperCorr_old.fits'

table = Table.read(table_path)
print(table[:5])
table.info()
print(table.columns)

Now let's choose a galaxy, read its aperture data and overplot it!

In [ ]:
def load_cutout(file_path, index=1):
    """Loads a FITS cutout file and extracts the data, header, and WCS."""
    try:
        with fits.open(file_path) as hdu:
            data = hdu[index].data
            header = hdu[index].header
            wcs = WCS(header)
        return data, wcs
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return None, None

def adjust_aperture(galaxy_id, filter, survey, obs, output_folder, save_plot=False):
    
    # --- Load the FITS table ---
    #table_path =  '/home/bpc/University/master/Red_Cardinal/Flux_Aperture_PSFMatched_AperCorr_old.fits'
    #aperture_table = Table.read(table_path)
    table_path =  '/home/bpc/University/master/Red_Cardinal/aperture_table.csv'
    df = pd.read_csv(table_path)
    
    # --- Select the galaxy by ID ---
    #row = aperture_table[aperture_table['ID'] == galaxy_id][0]
    galaxy_id = int(galaxy_id)
    row = df[df['ID'] == galaxy_id].iloc[0]

    # --- Read in rotation angle of MIRI FITS file ---
    angle_file = '/home/bpc/University/master/Red_Cardinal/rotation_angles.json'
    with open(angle_file, "r") as f:
        angles = json.load(f)
    angle = angles[f"angle_{survey}{obs}"]
    
    # --- Read WCS from NIRCam image ---
    nircam_path = f"/home/bpc/University/master/Red_Cardinal/NIRCam/F444W_cutouts/{galaxy_id}_F444W_cutout.fits"
    nircam_data, nircam_wcs = load_cutout(nircam_path)

    # --- Convert NIRCam pixel coordinates to sky ---
    sky_coord = nircam_wcs.pixel_to_world(row['Apr_Xcenter'], row['Apr_Ycenter'])
    
    # --- Open MIRI cutout image ---
    miri_path = f"/home/bpc/University/master/Red_Cardinal/cutouts/{galaxy_id}_{filter}_cutout_{survey}{obs}.fits"
    miri_data, miri_wcs = load_cutout(miri_path)

    # --- Convert sky coords to MIRI pixel coordinates ---
    miri_x, miri_y = miri_wcs.world_to_pixel(sky_coord)

    # --- Create elliptical region in MIRI pixel space ---
    nircam_scale = 0.03    # arcsec/pixel
    miri_scale = 0.11092  # arcsec per pixel
    
    # arcsec/pixel
    scale_factor = nircam_scale / miri_scale
    
    # --- Specify parameters for the ellips ---
    width = row['Apr_A'] * scale_factor
    height = row['Apr_B'] * scale_factor
    theta = -row['Apr_Theta']
    theta_new = ((theta - angle) % 180) * u.deg
    
    # --- Create region file and check if folder exists ---
    os.makedirs(output_folder, exist_ok=True)
    reg_file = os.path.join(output_folder, f'regions/{galaxy_id}_{survey}{obs}_aperture.reg') 
    
    # --- Write to DS9-compatible region file ---
    with open(reg_file, "w") as fh:
        fh.write("# Region file format: DS9 version 4.1\n")
        fh.write("global color=red dashlist=8 3 width=2 font=\"helvetica 10 normal\" "
                "select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1\n")
        fh.write("image\n")
        fh.write(f"ellipse({miri_x:.2f},{miri_y:.2f},{width:.2f},{height:.2f},{theta_new:.2f})\n")

    if save_plot:
        
        # --- Clean and prepare the MIRI data for plotting ---
        miri_clean = np.copy(miri_data)
        finite_vals = miri_clean[np.isfinite(miri_clean)].flatten()
        
        # Sort and get lowest 80% values
        sorted_vals = np.sort(finite_vals)
        cutoff_index = int(0.8 * len(sorted_vals))
        background_vals = sorted_vals[:cutoff_index]
        background_mean = np.mean(background_vals)

        # Replace NaNs or infs with background mean
        miri_clean[~np.isfinite(miri_clean)] = background_mean

        fig, ax = plt.subplots(figsize=(6,6))
        ax.imshow(miri_clean, origin='lower', cmap='gray', vmin=np.percentile(miri_clean, 5), vmax=np.percentile(miri_clean, 99))

        # Original ellipse (without rotation correction)
        ellipse_original = Ellipse(
            xy=(miri_x, miri_y),
            width=width,
            height=height,
            angle=theta,  # Just the original θ!
            edgecolor='red',
            facecolor='none',
            lw=2,
            label='Original Ellipse'
        )
        ax.add_patch(ellipse_original)

        ellipse = Ellipse(
            xy=(miri_x, miri_y),
            width=width,
            height=height,
            angle=theta_new.to_value(u.deg),
            edgecolor='blue',
            linestyle='--',  # maybe dashed to differentiate
            facecolor='none',
            lw=2,
            label='Rotated Ellipse'
        )
        ax.add_patch(ellipse)
        
        ax.set_title(f"Galaxy {galaxy_id} - {filter} ({survey}{obs})")
        ax.set_xlim(miri_x - 30, miri_x + 30)
        ax.set_ylim(miri_y - 30, miri_y + 30)
        ax.legend(loc='upper right')
        
        # Save figure
        png_path = os.path.join(output_folder, f'masks/{galaxy_id}_{survey}{obs}_aperture_overlay.png')
        plt.savefig(png_path, dpi=150, bbox_inches='tight')
        plt.close(fig)
        
    # Collect modified aperture data
    aperture_info = {
    'Flux': np.nan,           # You don't have these yet
    'Flux_Err': np.nan,        
    'Image_Err': np.nan,       
    'Flux_BKG': np.nan,        
    'AB_Mag': np.nan,          
    'Flux_BKG_Err': np.nan,    
    'N_PIX': np.nan,           
    'Apr_A': width,            # Rescaled aperture
    'Apr_B': height,
    'Apr_Xcenter': miri_x,
    'Apr_Ycenter': miri_y,
    'Apr_Theta': theta_new.to_value(u.deg),
    'ID': galaxy_id
    }
    
    return aperture_info
    

Now let's try and call the function:

In [ ]:
cutout_dir = "/home/bpc/University/master/Red_Cardinal/cutouts/"
phot_dir = "/home/bpc/University/master/Red_Cardinal/photometry/"

# Get all FITS file paths
fits_files = glob.glob(os.path.join(cutout_dir, '*.fits'))

# Get the basenames of the FITS files
fits_fnames = [os.path.basename(f) for f in fits_files]

adjusted_apertures = []

for fname in fits_fnames:
    id = fname.split('_')[0]
    filter = fname.split('_')[1]
    if filter == 'F1800W': continue
    survey_obs = fname.split('_')[3]
    if '003' in survey_obs:
        survey = 'primer'
        obs = '003'
    elif '004' in survey_obs:
        survey = 'primer'
        obs = '004'
    elif '1' in survey_obs:
        survey = 'cweb'
        obs = '1'
    elif '2' in survey_obs:
        survey = 'cweb'
        obs = '2'
    else:
        print(f"Unknown survey and/or observation number for galaxy {id}:\n")
        print(survey_obs)
    
    # Call and collect results
    result = adjust_aperture(id, filter, survey, obs, phot_dir, save_plot=False)
    if result:
        adjusted_apertures.append(result)

# After loop: create a DataFrame
df_apertures = pd.DataFrame(adjusted_apertures)

df_path = '/home/bpc/University/master/Red_Cardinal/photometry/adjusted_apertures.csv'

# (optional) Save to CSV or integrate into photometry table
df_apertures.to_csv(df_path, index=False)

Now we can easily open any given FITS file with its corresponding ellipse region

In [ ]:
# --- Launch DS9 with the MIRI cutout and the overplotted aperture ---
region_dir = "/home/bpc/University/master/Red_Cardinal/photometry/regions/"
cutout_dir = "/home/bpc/University/master/Red_Cardinal/cutouts/"
phot_dir = "/home/bpc/University/master/Red_Cardinal/photometry/"

id = '20397'
filter = 'F770W'
survey_obs = 'cweb2'
cutout_path = os.path.join(cutout_dir, f'{id}_{filter}_cutout_{survey_obs}.fits')
reg_path = os.path.join(region_dir, f'{id}_{survey_obs}_aperture.reg')
subprocess.run(["ds9", cutout_path, "-regions", reg_path])


Let's check the table:

In [ ]:
table_path =  '/home/bpc/University/master/Red_Cardinal/Flux_Aperture_PSFMatched_AperCorr_old.fits'

table = Table.read(table_path)
print(table[:5])
table.info()

# Section to perform the actual photometry

First, let's set some parameters straight:

pixscale_arcsec = 0.11092  # arcsec per pixel

pix_area_sr = 2.89208962133982e-13  # from MIRI header


Functions

In [ ]:
# --- Functions ---

def get_psf(filter_name):
    """Read MIRI PSF file"""
    psf_file = f'/home/bpc/University/master/Red_Cardinal/WebbPSF/PSF_MIRI_{filter_name}.fits'
    with fits.open(psf_file) as psf:
        return psf[0].data

def get_aperture_params(galaxy_id, aperture_table):
    """Retrieve aperture parameters from the CSV table."""
    df = pd.read_csv(aperture_table)
    row = df[df['ID'] == int(galaxy_id)].iloc[0]
    return row['Apr_Xcenter'], row['Apr_Ycenter'], row['Apr_A'], row['Apr_B'], row['Apr_Theta'] * u.deg

def calculate_aperture_correction(psf_data, aperture_params):
    """Calculate aperture correction for given PSF and aperture."""
    aperture = EllipticalAperture(
        positions=(psf_data.shape[1] / 2, psf_data.shape[0] / 2),
        a=aperture_params['a'],
        b=aperture_params['b'],
        theta=aperture_params['theta']
    )
    total_flux = np.sum(psf_data)
    phot_table = aperture_photometry(psf_data, aperture)
    flux_in_aperture = phot_table['aperture_sum'][0]
    return total_flux / flux_in_aperture

def get_fluxes(data, bkg_map, error_map, bkg_rms_map, aperture):
    """Calculate flux and uncertainty from aperture photometry."""
    data_bkgsub = data - bkg_map
    
    phot_table = aperture_photometry(data_bkgsub, aperture, method='exact')
    flux = phot_table['aperture_sum'][0]    # in MJy/sr
    
    phot_table = aperture_photometry(bkg_map, aperture, method='exact')
    flux_bkg = phot_table['aperture_sum'][0]    # in MJy/sr
    
    # Mask for aperture
    aperture_mask = aperture.to_mask(method='exact')
    mask = aperture_mask.to_image(data_bkgsub.shape)
    
    # Flux uncertainty from ERR extension
    pixel_errors = error_map * mask
    flux_uncertainty_err = np.sqrt(np.sum(pixel_errors**2))
    
    # Background noise uncertainty in aperture from Background2D function
    flux_bkg_median = np.median(bkg_rms_map[mask>0])
    n_eff = np.sum(mask**2) / np.max(mask)**2  # approximate effective number of pixels
    flux_uncertainty_bkg = flux_bkg_median * np.sqrt(n_eff)
    
    # Total uncertainty
    flux_uncertainty = np.sqrt(flux_uncertainty_err**2 + flux_uncertainty_bkg**2)
    
    # Median error of the error map within the aperture
    median_error = np.median(error_map[mask>0])    
    
    # Convert everything to from MJy/sr to Jy
    miri_scale = 0.11092  # arcsec per pixel
    miri_scale_rad = miri_scale / 206265
    omega_pix = miri_scale_rad**2
    
    conv_factor = 1e6 * omega_pix
    
    flux *= conv_factor
    flux_bkg *= conv_factor
    flux_uncertainty *= conv_factor
    median_error *= conv_factor

    # Now everything is in Jy!!
    return flux, flux_uncertainty, flux_bkg, flux_uncertainty_bkg, median_error, n_eff

def get_background(galaxy_id, filter, data, aperture_params, sigma):
    """Subtract background using a sigma clipping."""
    aperture = EllipticalAperture(
        positions=(aperture_params['x_center'], aperture_params['y_center']),
        a=aperture_params['a'],
        b=aperture_params['b'],
        theta=aperture_params['theta']
    )
    
    # Define the clipping threshold
    sigma_clip = SigmaClip(sigma)
    
    # Create aperture mask (boolean, True = masked)
    apr_mask = aperture.to_mask(method='center').to_image(data.shape)
    apr_mask_bool = apr_mask.astype(bool)
    
    # Perform 2D background modelling
    bkg = Background2D(
        data,
        box_size=(10, 10),
        filter_size=(1, 1),  # smoothing kernel
        sigma_clip=sigma_clip,
        bkg_estimator=MedianBackground(),
        mask=apr_mask_bool
    )
    
    # Plot the clipped data with contours
    #plt.imshow(bkg.background, origin='lower', cmap='gray')
    #plt.contour(apr_mask_bool, levels=[0.5], colors='red')
    #plt.title(f'{galaxy_id} in {filter}')
    #plt.show()
    
    return bkg.background, bkg.background_rms, apr_mask_bool
    
def save_for_plotting(filename, original_data, bkg_map, aperture_params):
    """Save original data and sigma-clipped data to a FITS file, with aperture info in header."""
    
    # Create a Primary HDU
    hdu_primary = fits.PrimaryHDU(original_data)
    
    # Add aperture information into header
    hdr = hdu_primary.header
    hdr['Apr_Xcenter'] = aperture_params['x_center']
    hdr['Apr_Ycenter'] = aperture_params['y_center']
    hdr['Apr_A'] = aperture_params['a']
    hdr['Apr_B'] = aperture_params['b']
    hdr['Apr_Theta'] = aperture_params['theta']*180/np.pi
    
    # Extension HDU: sigma-clipped data (fill masked values with NaN)
    hdu_bkg = fits.ImageHDU(data=bkg_map, name='BKG')

    # Create HDU list and write to file
    hdul = fits.HDUList([hdu_primary, hdu_bkg])
    hdul.writeto(filename, overwrite=True)

# --- Main Loop ---

def perform_photometry(cutout_files, aperture_table, psf_data, output_folder, sigma):
    """Main loop for photometry analysis."""
    photometry_results = []
    
    for fits_path in cutout_files:
        
        fits_name = os.path.basename(fits_path)
        galaxy_id = fits_name.split('_')[0]
        filter = fits_name.split('_')[1]
        
        with fits.open(fits_path) as hdul:
            data = hdul['SCI'].data if 'SCI' in hdul else hdul[1].data
            error = hdul['ERR'].data if 'ERR' in hdul else hdul[2].data
            
        # Aperture parameters
        miri_x, miri_y, width, height, theta = get_aperture_params(galaxy_id, aperture_table)
        aperture_params = {'x_center': miri_x, 'y_center': miri_y, 'a': width / 2, 'b': height / 2, 'theta': theta.to_value(u.rad)}
        aperture = EllipticalAperture((miri_x, miri_y), a=width / 2, b=height / 2, theta=theta.to_value(u.rad))
        
        # Background subtraction
        if galaxy_id in ['12332', '12282']: sigma = 2.5
        bkg_map, bkg_rms_map, apr_mask = get_background(galaxy_id, filter, data, aperture_params, sigma=sigma)
        
        # Store data in a FITS file for later plotting
        plot_folder = os.path.join(output_folder, f'mosaic_fits/')
        os.makedirs(plot_folder, exist_ok=True)
        filename = os.path.join(plot_folder, f'{galaxy_id}_{filter}.fits')
        save_for_plotting(filename, data, bkg_map, aperture_params)        
        
        # Photometry
        flux, flux_err, flux_bkg, flux_bkg_err, median_error, n_pix = get_fluxes(data, bkg_map, error, bkg_rms_map, aperture)

        # Aperture correctionbkg_rms_map
        correction_factor = calculate_aperture_correction(psf_data, aperture_params)

        # Apply aperture correction
        flux_corrected = flux * correction_factor
        flux_err_corrected = flux_err * correction_factor
        flux_bkg_corrected = flux_bkg * correction_factor
        flux_bkg_err_corrected =  flux_bkg_err * correction_factor
        
        # AB magnitude
        if flux_corrected > 0:
            ab_mag = -2.5 * np.log10(flux_corrected) + 8.90
        else: ab_mag = np.nan
        
        # Append results
        photometry_results.append({
            'Flux': flux_corrected,
            'Flux_Err': flux_err_corrected,
            'Image_Err': median_error,
            'Flux_BKG': flux_bkg_corrected,
            'AB_Mag': ab_mag,
            'Flux_BKG_Err': flux_bkg_err_corrected,
            'N_PIX': n_pix,
            'Apr_A': width,
            'Apr_B': height,
            'Apr_Xcenter': miri_x,
            'Apr_Ycenter': miri_y,
            'Apr_Theta': theta.to_value(u.deg),
            'ID': int(galaxy_id)
        })
    
    # Save to output table (assuming it's a pandas DataFrame)
    os.makedirs(output_folder, exist_ok=True)
    output_table = os.path.join(output_folder, f'results/photometry_table_{filter}.csv')
    output_df = pd.DataFrame(photometry_results)
    output_df.to_csv(output_table, index=False)
    


Now here is where the magic happens!

I wanna call the photometry function with:

perform_photometry(galaxy_id, filter, aperture_table, psf_data, cutouts_folder, output_table)


In [ ]:
# --- Parameters ---
cutouts_folder = "/home/bpc/University/master/Red_Cardinal/cutouts/"
output_folder = '/home/bpc/University/master/Red_Cardinal/photometry/'
aperture_table = '/home/bpc/University/master/Red_Cardinal/photometry/adjusted_apertures.csv'
os.makedirs(output_folder, exist_ok=True)

#f770w_files = glob.glob(os.path.join(cutouts_folder, '*F770W*.fits'))
#f1800w_files = glob.glob(os.path.join(cutouts_folder, '*F1800W*.fits'))

f770w_files = []
f1800w_files = []

test_ids = ['8465', '9871', '12202', '8843', '7904', '8338', '10021', '10245']
for id in test_ids:
    file = glob.glob(os.path.join(cutouts_folder, f'{id}_F770W*.fits'))[0]
    f770w_files.append(file)
    if glob.glob(os.path.join(cutouts_folder, f'{id}_F1800W*.fits')) != []:
        file = glob.glob(os.path.join(cutouts_folder, f'{id}_F1800W*.fits'))[0]
        f1800w_files.append(file)

print(f770w_files)
print(f1800w_files)

psf_f770w = get_psf('F770W')
psf_f1800w = get_psf('F1800W')

perform_photometry(f770w_files, aperture_table, psf_f770w, output_folder, sigma=3.0)
perform_photometry(f1800w_files, aperture_table, psf_f1800w, output_folder, sigma=3.0)

Now let's try and create those juicy looking mosaics with the cutouts

In [ ]:
def plot_mosaic(filename):
    """Plot original data with aperture and clipped background map from a custom FITS file."""
    
    galaxy_id = os.path.basename(filename).split('_')[0]
    
    # Open FITS file
    with fits.open(filename) as hdul:
        # Extract data
        original_data = hdul[0].data
        bkg_clipped_data = hdul['BKG'].data
        bkgsub_data = original_data - bkg_clipped_data
        
        # Extract aperture parameters from header
        hdr = hdul[0].header
        x0 = hdr['Apr_Xcenter']
        y0 = hdr['Apr_Ycenter']
        a = hdr['Apr_A']
        b = hdr['Apr_B']
        theta = hdr['Apr_Theta']
    
    # Set up plotting
    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(6, 10), constrained_layout=True)

    # First plot: original image with aperture
    ax = axes[0]
    norm = simple_norm(original_data, 'sqrt', percent=99.)
    im = ax.imshow(original_data, origin='lower', cmap='magma', norm=norm)
    ellipse = Ellipse((x0, y0), width=2*a, height=2*b, angle=theta,
                      edgecolor='red', facecolor='none', lw=2)
    ax.add_patch(ellipse)
    plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    
    # Second plot: background subtracted image
    ax = axes[1]
    norm_bkg = simple_norm(bkgsub_data, 'sqrt', percent=99.)
    im_bkg = ax.imshow(bkgsub_data, origin='lower', cmap='magma', norm=norm_bkg)
    ellipse = Ellipse((x0, y0), width=2*a, height=2*b, angle=theta,
                      edgecolor='red', facecolor='none', lw=2)
    ax.add_patch(ellipse)
    ax.set_title(f'{galaxy_id} with boxsize (10,10) and smoothing=(1,1)')
    plt.colorbar(im_bkg, ax=ax, fraction=0.046, pad=0.04)
    
    # Third plot: sigma-clipped background map
    ax = axes[2]
    norm_bkg = simple_norm(bkg_clipped_data, 'linear', percent=99.)
    im_bkg = ax.imshow(bkg_clipped_data, origin='lower', cmap='viridis', norm=norm_bkg)
    ellipse = Ellipse((x0, y0), width=2*a, height=2*b, angle=theta,
                      edgecolor='red', facecolor='none', lw=2)
    ax.add_patch(ellipse)
    plt.colorbar(im_bkg, ax=ax, fraction=0.046, pad=0.04)
    
    out_path = f'/home/bpc/University/master/Red_Cardinal/photometry/bkg_sub_trials/{galaxy_id}_v3.png'
    plt.savefig(out_path, dpi=150, bbox_inches='tight')
    plt.close(fig)
    plt.show()

In [ ]:
fits_dir = '/home/bpc/University/master/Red_Cardinal/photometry/mosaic_fits/'
plot_files = glob.glob(os.path.join(fits_dir, '*.fits'))
#fits_file = os.path.join(fits_dir, '22199_F770W_masked.fits')
for fits_file in plot_files:
    plot_mosaic(fits_file)

Check the contents of the original FITS files:

In [ ]:
fname = './../cutouts/7102_F770W_cutout_cweb1.fits'
with fits.open(fname) as hdul:
    hdul.info()


fname = './../MIRI/PRIMER_003/jw01837-o003_t003_miri_f770w_i2d.fits'
with fits.open(fname) as hdul:
    hdul.info()


Let's consult Amir's table again and extract all fluxes for HST/NIRCam

In [ ]:
def plot_sed_with_cutouts(test_ids):

    # --- Loading the photometric data ---
    
    # Load FITS table from Amir
    table_path = '/home/bpc/University/master/Red_Cardinal/Flux_Aperture_PSFMatched_AperCorr_old.fits'
    tbl = Table.read(table_path, hdu=1)
    
    # Load MIRI CSV photometry tables
    f770w_data = pd.read_csv('/home/bpc/University/master/Red_Cardinal/photometry/results/photometry_table_F770W.csv')
    f1800w_data = pd.read_csv('/home/bpc/University/master/Red_Cardinal/photometry/results/photometry_table_F1800W.csv')
    f770w_data['ID'] = f770w_data['ID'].astype(str)
    f1800w_data['ID'] = f1800w_data['ID'].astype(str)

    # Define NIRCam/HST wavelengths
    nircam_lambdas = [0.7, 0.9, 1.15, 1.5, 2.0, 2.77, 3.56, 4.44]  # in microns

    # --- Load aperture info ---
    aperture_table = pd.read_csv('/home/bpc/University/master/Red_Cardinal/photometry/adjusted_apertures.csv')
    aperture_table['ID'] = aperture_table['ID'].astype(str)
    
    # --- Loading the cutouts ---
    cutout_dir = '/home/bpc/University/master/Red_Cardinal/cutouts/'
    
    # Convert IDs to string
    id_strings = [id.decode() if isinstance(id, bytes) else str(id) for id in tbl['ID']]

    # Create a boolean mask to select matching IDs
    mask = [id_str in test_ids for id_str in id_strings]

    # Apply the mask to get the rows of interest
    selected_rows = tbl[mask]

    
    for row in selected_rows:
        galaxy_id = row['ID'].decode() if isinstance(row['ID'], bytes) else row['ID']
        print(f"Processing Galaxy ID: {galaxy_id}")
        
        
        
        # --- Prepare NIRCam/HST photometric data ---
        ab_mag_array = row['AB_Mag']  # likely a numpy array or similar
        ab_mags_nircam = []
        wavelengths_nircam = []

        for lam, mag in zip(nircam_lambdas, ab_mag_array):
            if np.isfinite(mag) and mag < 90:
                wavelengths_nircam.append(lam)
                ab_mags_nircam.append(mag)
        
        
        
        # --- Prepare MIRI photometric data ---
        wavelengths_miri = []
        ab_mags_miri = []
        
        has_f770w = False
        has_f1800w = False
        
        # Extract MIRI F770W
        if galaxy_id in f770w_data['ID'].values:
            miri_row = f770w_data[f770w_data['ID'] == galaxy_id].iloc[0]
            mag = miri_row.get('AB_Mag')
            if np.isfinite(mag) and mag < 90:
                wavelengths_miri.append(7.7)
                ab_mags_miri.append(mag)
                has_f770w = True
                f770w_cutout = glob.glob(os.path.join(cutout_dir, f'{galaxy_id}_F770W_*.fits'))[0]


        # Extract MIRI F1800W
        if galaxy_id in f1800w_data['ID'].values:            
            miri_row = f1800w_data[f1800w_data['ID'] == galaxy_id].iloc[0]
            mag = miri_row.get('AB_Mag')
            if np.isfinite(mag) and mag < 90:
                wavelengths_miri.append(18.0)
                ab_mags_miri.append(mag)    
                has_f1800w = True
                f1800w_cutout = glob.glob(os.path.join(cutout_dir, f'{galaxy_id}_F1800W_*.fits'))[0]


        
        # --- Setting up the Aperture ---
        aperture_row = aperture_table[aperture_table['ID'] == galaxy_id]
        if aperture_row.empty:
            print(f"No aperture info for {galaxy_id}. Skipping.")
            continue

        a = aperture_row['Apr_A'].values[0]
        b = aperture_row['Apr_B'].values[0]
        x0 = aperture_row['Apr_Xcenter'].values[0]
        y0 = aperture_row['Apr_Ycenter'].values[0]
        theta = aperture_row['Apr_Theta'].values[0]
        
        
        
        # --- Plot setup ---
        ncols = 2 + int(has_f1800w)
        fig, axes = plt.subplots(1, ncols, figsize=(5 * ncols, 5))
        
        
        # Plot SED
        ax = axes[0]
        ax.scatter(wavelengths_nircam, ab_mags_nircam, label='NIRCam/HST', color='royalblue')
        ax.scatter(wavelengths_miri, ab_mags_miri, label='MIRI', color='darkorange')
        ax.invert_yaxis()
        ax.set_xlabel('Wavelength (μm)')
        ax.set_ylabel('AB Magnitude')
        ax.set_title(f"Galaxy {galaxy_id} SED")
        ax.grid(True)
        ax.legend()
        
        
        
        # --- Define a plot function within a plot function (I guess?) ---
        def plot_cutout(ax, fits_path, title):
            with fits.open(fits_path) as hdul:
                image = hdul['SCI'].data
            ax.imshow(image, origin='lower', cmap='magma')
            ellipse = Ellipse((x0, y0), width=2*a, height=2*b, angle=np.degrees(theta),
                              edgecolor='yellow', facecolor='none', lw=2)
            ax.add_patch(ellipse)
            ax.set_title(title)
            ax.axis('off')

        # Plot F770W cutout
        if has_f770w:
            plot_cutout(axes[1], f770w_cutout, 'F770W Cutout')

        # Plot F1800W cutout if available
        if has_f1800w:
            plot_cutout(axes[2], f1800w_cutout, 'F1800W Cutout')

        plt.tight_layout()
        plt.show()


Call the plotting function

In [ ]:
# Define test galaxy IDs
test_ids = ['8465', '9871', '12202', '8843', '7904', '8338', '10021', '10245']

plot_sed_with_cutouts(test_ids)

Let's perform a sanity check

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Wavelengths for HST/NIRCam filters (example values, in microns – adjust if needed)
nircam_wavelengths = np.array([0.9, 1.15, 1.5, 2.0, 2.7, 3.5, 4.0, 4.4])

# Example AB magnitudes from your table for 4 galaxies
ab_mags_dict = {
    '9871': [24.6426, 23.2576, 22.2273, 21.4171, 20.6848, 20.1994, 19.9901, 19.9017],
    '8843': [25.7030, 25.4981, 24.8838, 24.5870, 24.6189, 24.6943, 24.6097, 24.6236],
    '12202': [24.9202, 24.9578, 24.8138, 24.4899, 24.9881, 24.9117, 24.8796, 24.8480],
    '8465': [25.2414, 25.9663, 25.2447, 25.0359, 25.6392, 25.8209, 25.5421, 25.7917],
    '10245': [23.7153, 22.7920, 22.2495, 21.6660, 21.3146, 21.0996, 21.0346, 20.9307],
    '10021': [24.9202, 24.9578, 24.8138, 24.4899, 24.9881, 24.9117, 24.8796, 24.8480],
    '7904': [25.4541, 25.4538, 25.1352, 24.3570, 23.8030, 23.5368, 23.3419, 23.2143],
    '8338': [23.0856, 23.5154, 23.3476, 23.2422, 23.7255, 23.7303, 23.5780, 23.5289]
}

# Example MIRI photometric points (AB magnitudes + filter wavelengths)
miri_data = {
    '9871': [(7.7, 20.09564191328481), (18.0, 17.496047881291325)],
    '8843': [(7.7, 24.466914961995812)],
    '12202': [(7.7, 24.243698695608288)],
    '8465': [(7.7, 24.727584921067702), (18.0, 21.70948515662667)],
    '7904': [(7.7, 22.4914), (18.0, 21.3182)],
    '8338': [(7.7, 22.6910), (18.0, 21.3048)],
    '10021': [(7.7, 22.4679), (18.0, 19.3946)],
    '10245': [(7.7, 20.7895), (18.0, 16.9995)]
}

# Plotting
for galaxy_id, nircam_mags in ab_mags_dict.items():
    plt.figure(figsize=(6, 4))
    
    # Plot NIRCam photometry
    plt.scatter(nircam_wavelengths, nircam_mags, label='NIRCam', marker='o', color='tab:blue')

    # Plot MIRI photometry if available
    if galaxy_id in miri_data:
        miri_wavelengths, miri_mags = zip(*miri_data[galaxy_id])
        plt.scatter(miri_wavelengths, miri_mags, label='MIRI', marker='s', color='tab:orange')

    plt.gca().invert_yaxis()  # AB magnitude scale: brighter = lower
    plt.xlabel('Wavelength [µm]')
    plt.ylabel('AB Magnitude')
    plt.title(f'Galaxy ID: {galaxy_id}')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()
